# All the parameters that need to be changed

In [1]:
# Minnesota
state_ab = "mn"

## Data
1. Download all the data in directory "il_data"
2. Eextract them all

In [2]:
data_folder = state_ab + "_data/"
population1_data = "./{}{}_pl2020_b/{}_pl2020_p1_b.shp".format(data_folder, state_ab, state_ab)
population2_data = "./{}{}_pl2020_b/{}_pl2020_p2_b.shp".format(data_folder, state_ab, state_ab)
vap_data =  "./{}{}_pl2020_b/{}_pl2020_p4_b.shp".format(data_folder, state_ab, state_ab)
vest20_data = "./{}{}_vest_20/{}_vest_20.shp".format(data_folder, state_ab, state_ab)
vest18_data = "./{}{}_vest_18/{}_vest_18.shp".format(data_folder, state_ab, state_ab)
vest16_data = "./{}{}_vest_16/{}_vest_16.shp".format(data_folder, state_ab, state_ab)
cd_data = "./{}{}_cong_adopted_2022/c2022.shp".format(data_folder, state_ab)
send_data = "./{}{}_sldu_adopted_2022/l2022-sen-shp.shp".format(data_folder, state_ab)
hdist_data = "./{}{}_sldl_adopted_2022/l2022_hse.shp".format(data_folder, state_ab)

## Parameters that needs to be manually checked

### base vest data
start_col = 5\
vest_base_data = vest20\
year = '20'

### district data
district column name of cong_df, send, hdist when calling add_dist()

# Program starts

In [3]:
import pandas as pd
import geopandas as gpd
import maup
import time
from maup import smart_repair
from gerrychain import Graph
import os

maup.progress.enabled = True

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
def do_smart_repair(df):
    # change it to the UTM it needs for smart_repair
    df = df.to_crs(df.estimate_utm_crs())
    df = smart_repair(df)
    
    # check maup doctor again to see if smart repair works
    if maup.doctor(df) == True:
        # change it back to this UTM for this data
        df = df.to_crs('EPSG:4269')
    else:
        raise Exception('maup.doctor failed')
    
    return df

In [6]:
def add_district(dist_df, dist_name, election_df, col_name):
    # check if it needs to be smart_repair
    if maup.doctor(dist_df) != True:
        dist_df = do_smart_repair(dist_df)

    election_df = gpd.GeoDataFrame(election_df, crs="EPSG:4269")

    # assigne the pricincts
    precincts_to_district_assignment = maup.assign(election_df.geometry, dist_df.geometry)
    election_df[dist_name] = precincts_to_district_assignment
    for precinct_index in range(len(election_df)):
        election_df.at[precinct_index, dist_name] = dist_df.at[election_df.at[precinct_index, dist_name], col_name]

    return election_df

In [7]:
def rename(original, year):
    party = original[6]
    if party == 'R' or party == 'D':
        return original[3:6] + year + original[6]
    else:
        return original[3:6] + year + 'O'

In [8]:
def check_population(population, df):
    pop_check = pd.DataFrame({
        'pop_col': pop_col,
        'population_df': population[pop_col].sum(), 
        'vest_base': df[pop_col].sum(),
        'equal': [x == y for x, y in zip(population[pop_col].sum(), df[pop_col].sum())]
    })
    if pop_check['equal'].mean() < 1:
        print(pop_check)
        raise Exception("population doesn't agree")

    else:
        print("population agrees")

In [9]:
def add_vest(vest, df, year, population, start_col):    
     # check if it needs to be smart_repair
    if maup.doctor(vest) != True:
        vest = do_smart_repair(vest)
    
    # rename the columns
    original_col = vest.columns[start_col:-1]
    new_col = [rename(i, year) for i in original_col]
    rename_dict = dict(zip(original_col, new_col))
    vest = vest.rename(columns=rename_dict)
    vest = vest.groupby(level=0, axis=1).sum() # combine all the other party's vote into columns with sufix "O"
    col_name = list(set(new_col))
    col_name.sort()
    
    # make the blocks from precincts by weight
    vest = gpd.GeoDataFrame(vest, crs="EPSG:4269")
    election_in_block = population[["VAP", 'geometry']] # population_df is in block scale
    blocks_to_precincts_assignment = maup.assign(election_in_block.geometry, vest.geometry)
    weights = election_in_block["VAP"] / blocks_to_precincts_assignment.map(election_in_block["VAP"].groupby(blocks_to_precincts_assignment).sum())
    weights = weights.fillna(0)
    prorated = maup.prorate(blocks_to_precincts_assignment, vest[col_name], weights)
    election_in_block[col_name] = prorated
    
    # assign blocks to precincts
    election_in_block = gpd.GeoDataFrame(election_in_block, crs="EPSG:4269")
    df = gpd.GeoDataFrame(df, crs="EPSG:4269")
    block_to_pricinct_assginment = maup.assign(election_in_block.geometry, df.geometry)
    df[col_name] = election_in_block[col_name].groupby(block_to_pricinct_assginment).sum()
    df = df.groupby(level=0, axis=1).sum()
    
    # check if population agrees
    check_population(population, df)
    
    return df

## Read the census data

In [10]:
population1_df = gpd.read_file(population1_data)
population2_df = gpd.read_file(population2_data)
vap_df = gpd.read_file(vap_data)

In [11]:
population2_df = population2_df.drop(columns=['SUMLEV', 'LOGRECNO', 'GEOID', 'COUNTY', 'geometry'])
vap_df = vap_df.drop(columns=['SUMLEV', 'LOGRECNO', 'GEOID', 'COUNTY', 'geometry'])

In [12]:
population_df = pd.merge(population1_df, population2_df, on='GEOID20')
population_df = pd.merge(population_df, vap_df, on='GEOID20')

In [13]:
rename_dict = {'P0020001': 'TOTPOP', 'P0020002': 'HISP', 'P0020005': 'NH_WHITE', 'P0020006': 'NH_BLACK', 'P0020007': 'NH_AMIN',
               'P0020008': 'NH_ASIAN', 'P0020009': 'NH_NHPI', 'P0020010': 'NH_OTHER', 'P0020011': 'NH_2MORE',
               'P0040001': 'VAP', 'P0040002': 'HVAP', 'P0040005': 'WVAP', 'P0040006': 'BVAP', 'P0040007': 'AMINVAP',
               'P0040008': 'ASIANVAP', 'P0040009': 'NHPIVAP', 'P0040010': 'OTHERVAP', 'P0040011': '2MOREVAP'}

In [14]:
population_df.rename(columns=rename_dict, inplace = True)

In [15]:
population_df['H_WHITE'] = population_df.apply(lambda t: t['P0010003'] - t['NH_WHITE'], 1)
population_df['H_BLACK'] = population_df.apply(lambda t: t['P0010004'] - t['NH_BLACK'], 1)
population_df['H_AMIN'] = population_df.apply(lambda t: t['P0010005'] - t['NH_AMIN'], 1)
population_df['H_ASIAN'] = population_df.apply(lambda t: t['P0010006'] - t['NH_ASIAN'], 1)
population_df['H_NHPI'] = population_df.apply(lambda t: t['P0010007'] - t['NH_NHPI'], 1)
population_df['H_OTHER'] = population_df.apply(lambda t: t['P0010008'] - t['NH_OTHER'], 1)
population_df['H_2MORE'] = population_df.apply(lambda t: t['P0010009'] - t['NH_2MORE'], 1)

# Read the base vest data
Now using it as a "base precinct", but it could be vest 18 or vest 16 if vest 20 is not working

In [16]:
def add_vest_base(vest, start_col, year):
    original_col = vest.columns[start_col:-1]
    new_col = [rename(i, year) for i in original_col]
    rename_dict = dict(zip(original_col, new_col))
    vest = vest.rename(columns=rename_dict)
    vest = vest.groupby(level=0, axis=1).sum()
    vest = gpd.GeoDataFrame(vest, crs="EPSG:4269")
    
    return vest

### Check if vest 20 can be used as base

In [17]:
vest20 = gpd.read_file(vest20_data)

In [18]:
if maup.doctor(vest20) != True:
    vest20 = do_smart_repair(vest20)

100%|████████████████████████████████████████| 4110/4110 [00:06<00:00, 682.90it/s]


There are some invalid geometries.
Snapping all geometries to a grid with precision 10^( -5 ) to avoid GEOS errors.
Identifying overlaps...


100%|███████████████████████████████████████| 4932/4932 [00:02<00:00, 2178.66it/s]


Resolving overlaps...
Filling gaps...


Gaps to simplify: 0it [00:00, ?it/s]
Gaps to fill: 0it [00:00, ?it/s]
100%|████████████████████████████████████████| 4110/4110 [00:05<00:00, 821.40it/s]


### If it is true for maup doctor, we will use it as the base vest data.
Check where the election column starts, this should be the same for all vest data in that state

In [50]:
list(vest20.columns)

['VTDID',
 'PCTNAME',
 'COUNTYNAME',
 'COUNTYFIPS',
 'G20PREDBID',
 'G20PRERTRU',
 'G20PRELJOR',
 'G20PREGHAW',
 'G20PREAFUE',
 'G20PREPLAR',
 'G20PRESKEN',
 'G20PREIWES',
 'G20PREIPIE',
 'G20PREOWRI',
 'G20USSDSMI',
 'G20USSRLEW',
 'G20USSMOCO',
 'G20USSCSTE',
 'G20USSOWRI',
 'geometry']

## Parameters that need to be checked

In [20]:
start_col = 4
vest_base_data = vest20
year = '20'

In [21]:
vest_base = add_vest_base(vest_base_data, start_col, year)

In [22]:
# vap and population have the same GEOID20
blocks_to_precincts_assignment = maup.assign(population_df.geometry, vest_base.geometry)

100%|█████████████████████████████████████████| 4110/4110 [00:48<00:00, 84.44it/s]


In [23]:
pop_col = ['TOTPOP', 'HISP', 'NH_WHITE', 'NH_BLACK', 'NH_AMIN', 'NH_ASIAN', 'NH_NHPI', 'NH_OTHER', 'NH_2MORE', 'H_WHITE', 'H_BLACK', 'H_AMIN', 'H_ASIAN', 'H_NHPI', 'H_OTHER', 'H_2MORE', 'VAP', 'HVAP', 'WVAP', 'BVAP', 'AMINVAP', 'ASIANVAP', 'NHPIVAP', 'OTHERVAP', '2MOREVAP']

In [24]:
vest_base[pop_col] = population_df[pop_col].groupby(blocks_to_precincts_assignment).sum()

In [25]:
election_df = gpd.GeoDataFrame(vest_base, crs="EPSG:4269")

### Check if the population agrees

In [26]:
check_population(population_df, vest_base)

population agrees


# Add more vest data

In [29]:
vest18 = gpd.read_file(vest18_data)

In [30]:
list(vest18.columns)

['COUNTYNAME',
 'COUNTYFIPS',
 'VTDID',
 'PCTNAME',
 'PCTCODE',
 'SHORTLABEL',
 'MCDNAME',
 'MCDCODE',
 'MCDFIPS',
 'MCDGNIS',
 'CONGDIST',
 'MNSENDIST',
 'MNLEGDIST',
 'CTYCOMDIST',
 'JUDDIST',
 'G18USSRNEW',
 'G18USSDKLO',
 'G18USSMSCH',
 'G18USSGOVE',
 'G18USSOWRI',
 'S18USSRHOU',
 'S18USSDSMI',
 'S18USSMWEL',
 'S18USSUTRO',
 'S18USSOWRI',
 'G18GOVRJOH',
 'G18GOVDWAL',
 'G18GOVMWRI',
 'G18GOVLWEL',
 'G18GOVOWRI',
 'G18SOSRHOW',
 'G18SOSDSIM',
 'G18SOSIDEN',
 'G18SOSOWRI',
 'G18AUDRMYH',
 'G18AUDDBLA',
 'G18AUDMFOR',
 'G18AUDLDOC',
 'G18AUDOWRI',
 'G18ATGRWAR',
 'G18ATGDELL',
 'G18ATGMJOH',
 'G18ATGOWRI',
 'geometry']

In [31]:
# check the result here
start_col = 15
election_df = add_vest(vest18, election_df, '18', population_df, start_col)

100%|████████████████████████████████████████| 4113/4113 [00:06<00:00, 630.83it/s]


There are some invalid geometries.
Snapping all geometries to a grid with precision 10^( -5 ) to avoid GEOS errors.
Identifying overlaps...


100%|███████████████████████████████████████| 4957/4957 [00:03<00:00, 1646.11it/s]


Resolving overlaps...
Filling gaps...


Gaps to simplify: 0it [00:00, ?it/s]
Gaps to fill: 0it [00:00, ?it/s]
100%|█████████████████████████████████████████| 4110/4110 [00:46<00:00, 89.23it/s]


population agrees


In [32]:
vest16 = gpd.read_file(vest16_data)
vest16.columns

Index(['VTDID', 'PCTNAME', 'PCTCODE', 'SHORTLABEL', 'MCDNAME', 'MCDCODE',
       'MCDFIPS', 'MCDGNIS', 'G16PRERTru', 'G16PREDCli', 'G16PRECCas',
       'G16PREMVac', 'G16PRESKen', 'G16PREGSte', 'G16PREIDeL', 'G16PREIMcM',
       'G16PRELJoh', 'G16PREOth', 'geometry'],
      dtype='object')

In [33]:
start_col = 8
election_df = add_vest(vest16, election_df, '16', population_df, start_col)

100%|████████████████████████████████████████| 4120/4120 [00:06<00:00, 594.04it/s]


There are some invalid geometries.
Snapping all geometries to a grid with precision 10^( -5 ) to avoid GEOS errors.
Identifying overlaps...


100%|███████████████████████████████████████| 5021/5021 [00:02<00:00, 2157.91it/s]


Resolving overlaps...
Filling gaps...


Gaps to simplify: 0it [00:00, ?it/s]
Gaps to fill: 0it [00:00, ?it/s]
100%|█████████████████████████████████████████| 4110/4110 [00:50<00:00, 81.39it/s]


population agrees


## Add the district data

In [36]:
cong_df = gpd.read_file(cd_data).to_crs('EPSG:4269')
send = gpd.read_file(send_data).to_crs('EPSG:4269')
hdist = gpd.read_file(hdist_data).to_crs('EPSG:4269')

In [37]:
cong_df.head()

,ID,AREA,DISTRICT,MEMBERS,LOCKED,NAME,POPULATION,NH_WHT,HISPANIC_O,NH_DOJ_BLK,...,F_NH18_DO3,F_NH18_DO4,F_NH18_DO5,F_NH18_WHT,F_18_POP,F_DEM_INDE,F_REP_INDE,F_THIRD_IN,DISTRICT_L,geometry
0,1,12430.222656,1,1.0,None,None,713311,587883,51121,27369,...,0.000601,0.002295,0.022386,0.855693,0.770142,0.477860,0.474246,0.047894,1|-0%,"POLYGON ((-92.73204 44.71434, -92.72862 44.711..."
1,2,1805.995972,2,1.0,None,None,713312,536985,54131,44486,...,0.000441,0.004392,0.030024,0.790203,0.750402,0.494260,0.460461,0.045279,2|0%,"POLYGON ((-93.40457 44.19639, -93.40657 44.196..."
2,3,515.873108,3,1.0,None,None,713311,509159,37414,67558,...,0.000233,0.003791,0.030805,0.757537,0.769882,0.522974,0.441346,0.035680,3|-0%,"POLYGON ((-93.38967 45.24109, -93.38945 45.241..."
3,4,333.247986,4,1.0,None,None,713312,443863,52740,79285,...,0.000340,0.003779,0.033634,0.678897,0.770141,0.636964,0.311469,0.051567,4|0%,"POLYGON ((-93.02052 45.12387, -93.02009 45.123..."
4,5,136.886185,5,1.0,None,None,713312,427555,72377,121980,...,0.000393,0.004791,0.039950,0.653848,0.801732,0.748150,0.201100,0.050750,5|0%,"POLYGON ((-93.26578 45.12458, -93.26443 45.124..."


In [38]:
set(cong_df['DISTRICT'])

{'1', '2', '3', '4', '5', '6', '7', '8'}

In [39]:
election_df = add_district(cong_df, "CD", election_df, "DISTRICT")

100%|███████████████████████████████████████████████| 8/8 [00:12<00:00,  1.62s/it]


In [40]:
send.head()

,ID,AREA,DISTRICT,MEMBERS,LOCKED,NAME,POPULATION,NH_WHT,HISPANIC_O,NH_DOJ_BLK,...,F_NH18_DO3,F_NH18_DO4,F_NH18_DO5,F_NH18_WHT,F_18_POP,F_DEM_INDE,F_REP_INDE,F_THIRD_IN,DISTRICT_L,geometry
0,1,8599.527344,01,1.0,None,None,85175,75057,3788,1267,...,0.000277,0.002677,0.026662,0.904585,0.763135,0.457220,0.497637,0.045143,01|0%,"POLYGON ((-96.56409 47.15143, -96.56906 47.151..."
1,2,8808.734375,02,1.0,None,None,85461,59017,1748,595,...,0.000171,0.002957,0.046515,0.745592,0.751910,0.475356,0.474558,0.050086,02|0.34%,"POLYGON ((-94.43027 48.70092, -94.43023 48.699..."
2,3,14772.773438,03,1.0,None,None,84754,76968,1144,526,...,0.000175,0.002531,0.033038,0.921324,0.811030,0.538565,0.421804,0.039631,03|-0.49%,"POLYGON ((-94.43027 48.70092, -94.42416 48.705..."
3,4,1340.252563,04,1.0,None,None,85002,71651,3383,3281,...,0.000358,0.002150,0.035057,0.873516,0.755065,0.522808,0.426478,0.050714,04|-0.2%,"POLYGON ((-95.79480 46.71651, -95.80751 46.716..."
4,5,3758.213867,05,1.0,None,None,85402,77560,3313,383,...,0.000888,0.002020,0.026308,0.926662,0.765099,0.365531,0.592854,0.041615,05|0.27%,"POLYGON ((-95.14588 46.10676, -95.14589 46.107..."


In [41]:
len(set(send['DISTRICT']))

67

In [42]:
hdist.head()

,ID,AREA,DISTRICT,MEMBERS,LOCKED,NAME,POPULATION,NH_WHT,HISPANIC_O,NH_DOJ_BLK,...,F_NH18_DO3,F_NH18_DO4,F_NH18_DO5,F_NH18_WHT,F_18_POP,F_DEM_INDE,F_REP_INDE,F_THIRD_IN,DISTRICT_L,geometry
0,1,5197.243652,01A,1.0,None,None,42570,38247,1382,275,...,0.000550,0.002933,0.026548,0.918034,0.768922,0.433413,0.521210,0.045377,01A|-0.04%,"POLYGON ((-95.58216 47.93397, -95.58225 47.933..."
1,2,3402.283936,01B,1.0,None,None,42605,36810,2406,992,...,0.000000,0.002417,0.026777,0.890941,0.757352,0.482788,0.472321,0.044891,01B|0.04%,"POLYGON ((-96.56409 47.15143, -96.56906 47.151..."
2,3,4535.418457,02A,1.0,None,None,42871,30297,1067,510,...,0.000124,0.002412,0.039549,0.757297,0.754356,0.501386,0.444328,0.054286,02A|0.67%,"POLYGON ((-94.42870 48.36733, -94.42577 48.367..."
3,4,4273.315918,02B,1.0,None,None,42590,28720,681,85,...,0.000219,0.003509,0.053573,0.733732,0.749448,0.449601,0.504468,0.045931,02B|0.01%,"POLYGON ((-94.42361 48.01981, -94.42244 48.019..."
4,5,13688.615234,03A,1.0,None,None,42417,37934,611,240,...,0.000170,0.002830,0.032229,0.911208,0.833180,0.521834,0.434363,0.043803,03A|-0.4%,"POLYGON ((-94.43027 48.70092, -94.42416 48.705..."


In [43]:
len(set(hdist['DISTRICT']))

134

In [44]:
election_df = add_district(send, "SEND", election_df, "DISTRICT")

100%|█████████████████████████████████████████████| 67/67 [00:08<00:00,  7.84it/s]


In [45]:
election_df = add_district(hdist, "HDIST", election_df, "DISTRICT")

100%|███████████████████████████████████████████| 134/134 [00:08<00:00, 16.68it/s]


In [49]:
list(election_df.columns)

['2MOREVAP',
 'AMINVAP',
 'ASIANVAP',
 'ATG18D',
 'ATG18O',
 'ATG18R',
 'AUD18D',
 'AUD18O',
 'AUD18R',
 'BVAP',
 'COUNTYFIPS',
 'COUNTYNAME',
 'GOV18D',
 'GOV18O',
 'GOV18R',
 'HISP',
 'HVAP',
 'H_2MORE',
 'H_AMIN',
 'H_ASIAN',
 'H_BLACK',
 'H_NHPI',
 'H_OTHER',
 'H_WHITE',
 'NHPIVAP',
 'NH_2MORE',
 'NH_AMIN',
 'NH_ASIAN',
 'NH_BLACK',
 'NH_NHPI',
 'NH_OTHER',
 'NH_WHITE',
 'OTHERVAP',
 'PCTNAME',
 'PRE16D',
 'PRE16O',
 'PRE16R',
 'PRE20D',
 'PRE20O',
 'PRE20R',
 'SOS18D',
 'SOS18O',
 'SOS18R',
 'TOTPOP',
 'USS18D',
 'USS18O',
 'USS18R',
 'USS20D',
 'USS20O',
 'USS20R',
 'VAP',
 'VTDID',
 'WVAP',
 'geometry',
 'CD',
 'SEND',
 'HDIST']

### Put the base precinct year after the precinct information column

In [51]:
base_columns = {}
if 'COUNTYFIPS' + year not in election_df.columns:
    base_columns = {
        'COUNTYFIPS': 'COUNTYFIPS' + year,
        'COUNTYNAME': 'COUNTYNAME' + year,
        'VTDID': 'VTDID' + year,
        'PCTNAME': 'PCTNAME' + year,
    }
election_df.rename(columns=base_columns, inplace = True)

In [52]:
# reorder the columns
fixed_columns = [
    'COUNTYFIPS' + year,
    'COUNTYNAME' + year,
    'VTDID' + year,
    'PCTNAME' + year,
    'CD',
    'SEND',
    'HDIST',
    'TOTPOP',
    'NH_2MORE',
    'NH_AMIN',
    'NH_ASIAN',
    'NH_BLACK',
    'NH_NHPI',
    'NH_OTHER',
    'NH_WHITE',
    'HISP',
    'H_AMIN',
    'H_ASIAN',
    'H_BLACK',
    'H_NHPI',
    'H_OTHER',
    'H_WHITE',
    'H_2MORE',
    'VAP',
    'HVAP',
    'WVAP',
    'BVAP',
    'AMINVAP',
    'ASIANVAP',
    'NHPIVAP',
    'OTHERVAP',
    '2MOREVAP']

election_columns = [col for col in election_df.columns if col not in fixed_columns]
final_col = fixed_columns + election_columns
election_df = election_df[final_col]

In [53]:
os.makedirs("./{}".format(state_ab))
election_df.to_file("./{}/{}.shp".format(state_ab, state_ab))
election_df.to_file('./{}/{}.geojson'.format(state_ab, state_ab), driver='GeoJSON')

# Only do once to build json and read from file when generating ensembles
graph = Graph.from_file("./{}/{}.shp".format(state_ab, state_ab), ignore_errors=True)
graph.to_json("./{}/{}.json".format(state_ab, state_ab))